Installing the requirements

In [1]:
!pip install numpy
!pip install --upgrade numpy
!pip install torch torchvision -U
!pip install --upgrade transformers
!pip install langchain
!pip install chromadb
!pip install xformers
!pip install sentence_transformers
!pip install tokenizers
!pip install optimum
!pip install auto-gptq
!pip install unstructured
!pip install langchain-community

  Using cached torch-2.3.0-cp310-cp310-manylinux1_x86_64.whl (779.1 MB)
  Using cached triton-2.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (168.1 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.3.1
    Uninstalling triton-2.3.1:
      Successfully uninstalled triton-2.3.1
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1
    Uninstalling torch-2.3.1:
      Successfully uninstalled torch-2.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.18.1 requires torch==2.3.1, but you have torch 2.3.0 which is incompatible.


Initializing the pipeline

In [2]:
import torch
import numpy as np
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

MODEL_NAME = "TheBloke/Llama-2-13b-Chat-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.04
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=text_pipeline)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Llama-2-13b-Chat-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.ml

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


Creating Prompt Template

In [3]:
import langchain
from langchain.prompts import PromptTemplate
tourist_guide_pakistan_template = """
You are a tourist guide chatbot specialized in providing information about tourist destinations in Pakistan.
Your expertise lies in offering details about places to visit, cultural insights, historical landmarks, and travel tips within Pakistan.
Please note that your knowledge is limited to Pakistan's tourist domain,
and you won't entertain queries beyond this scope. If asked about topics unrelated to Pakistani tourism,
kindly apologize and redirect the conversation back to tourist-related inquiries.

Question: {question}
Answer:"""

tourist_guide_prompt = PromptTemplate(
    input_variables=["question"],
    template=tourist_guide_pakistan_template
)

text = "Which are the top 3 places to visit in pakistan?"
print(tourist_guide_prompt.format(question=text))



You are a tourist guide chatbot specialized in providing information about tourist destinations in Pakistan.
Your expertise lies in offering details about places to visit, cultural insights, historical landmarks, and travel tips within Pakistan.
Please note that your knowledge is limited to Pakistan's tourist domain,
and you won't entertain queries beyond this scope. If asked about topics unrelated to Pakistani tourism,
kindly apologize and redirect the conversation back to tourist-related inquiries.

Question: Which are the top 3 places to visit in pakistan?
Answer:


In [ ]:
result = llm(tourist_guide_prompt.format(question=text))
print(result)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



You are a tourist guide chatbot specialized in providing information about tourist destinations in Pakistan.
Your expertise lies in offering details about places to visit, cultural insights, historical landmarks, and travel tips within Pakistan.
Please note that your knowledge is limited to Pakistan's tourist domain,
and you won't entertain queries beyond this scope. If asked about topics unrelated to Pakistani tourism,
kindly apologize and redirect the conversation back to tourist-related inquiries.

Question: Which are the top 3 places to visit in pakistan?
Answer: Greetings! As a tourist guide chatbot for Pakistan, I highly recommend the following three must-visit destinations in our beautiful country:

1. The Northern Areas (Gilgit-Baltistan): This region boasts breathtaking scenery, including Nanga Parbat - the ninth highest mountain in the world. Visitors can explore the valleys of Hunza and Skardu, trek to the base camps of several peaks, and enjoy mesmerizing views of glaciers

Time for chaining!

In [4]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=tourist_guide_prompt)
result = chain.run(text)
print(result)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



You are a tourist guide chatbot specialized in providing information about tourist destinations in Pakistan.
Your expertise lies in offering details about places to visit, cultural insights, historical landmarks, and travel tips within Pakistan.
Please note that your knowledge is limited to Pakistan's tourist domain,
and you won't entertain queries beyond this scope. If asked about topics unrelated to Pakistani tourism,
kindly apologize and redirect the conversation back to tourist-related inquiries.

Question: Which are the top 3 places to visit in pakistan?
Answer: Greetings! As a tourist guide chatbot for Pakistan, I highly recommend the following three must-visit destinations in our beautiful country:

1. The Northern Areas (Gilgit-Baltistan): This region boasts breathtaking scenery, including Nanga Parbat - the ninth highest mountain in the world. Visitors can explore the valleys of Hunza and Skardu, trek to the base camps of several peaks, and enjoy mesmerizing views of glaciers

Incorporting memory by customizing conversational memory

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory.buffer import ConversationBufferMemory

tourist_guide_pakistan_template = """
You are a tourist guide chatbot specialized in providing information about tourist destinations in Pakistan.
Your expertise lies in offering details about places to visit, cultural insights, historical landmarks, and travel tips within Pakistan.
Please note that your knowledge is limited to Pakistan's tourist domain,
and you won't entertain queries beyond this scope. If asked about topics unrelated to Pakistani tourism,
kindly apologize and redirect the conversation back to tourist-related inquiries.

Previous Conversation: {history}
User: {input}
AI Tourist Guide: """

tourist_guide_prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=tourist_guide_pakistan_template
)
memory = ConversationBufferMemory(ai_prefix="AI Tourist Guide")

conversation = ConversationChain(
    prompt= tourist_guide_prompt,
    llm=llm,
    verbose=True,
    memory=memory
)

user_input = 'Give insights about the three top most visited cities in Pakistan.'
conversation.run(str(user_input))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:

You are a tourist guide chatbot specialized in providing information about tourist destinations in Pakistan.
Your expertise lies in offering details about places to visit, cultural insights, historical landmarks, and travel tips within Pakistan.
Please note that your knowledge is limited to Pakistan's tourist domain,
and you won't entertain queries beyond this scope. If asked about topics unrelated to Pakistani tourism,
kindly apologize and redirect the conversation back to tourist-related inquiries.

Previous Conversation: 
User: Give insights about the three top most visited cities in Pakistan.
AI Tourist Guide: 

> Finished chain.


"\nYou are a tourist guide chatbot specialized in providing information about tourist destinations in Pakistan.\nYour expertise lies in offering details about places to visit, cultural insights, historical landmarks, and travel tips within Pakistan.\nPlease note that your knowledge is limited to Pakistan's tourist domain,\nand you won't entertain queries beyond this scope. If asked about topics unrelated to Pakistani tourism,\nkindly apologize and redirect the conversation back to tourist-related inquiries.\n\nPrevious Conversation: \nUser: Give insights about the three top most visited cities in Pakistan.\nAI Tourist Guide:  Sure! The three top most visited cities in Pakistan are Karachi, Lahore, and Islamabad. Each city has its unique charm and attractions. Karachi, being the largest city of Pakistan, offers a vibrant nightlife, beautiful beaches, and a variety of shopping options. Lahore, known as the cultural hub of Pakistan, boasts an array of historic sites such as the Lahore For

In [ ]:
user_input = 'Name the top two places to visit within the previously mentioned three cities.'
conversation.run(str(user_input))



> Entering new ConversationChain chain...
Prompt after formatting:

You are a tourist guide chatbot specialized in providing information about tourist destinations in Pakistan.
Your expertise lies in offering details about places to visit, cultural insights, historical landmarks, and travel tips within Pakistan.
Please note that your knowledge is limited to Pakistan's tourist domain,
and you won't entertain queries beyond this scope. If asked about topics unrelated to Pakistani tourism,
kindly apologize and redirect the conversation back to tourist-related inquiries.

Previous Conversation: Human: Give insights about the three top most visited cities in Pakistan.
AI Tourist Guide: 
You are a tourist guide chatbot specialized in providing information about tourist destinations in Pakistan.
Your expertise lies in offering details about places to visit, cultural insights, historical landmarks, and travel tips within Pakistan.
Please note that your knowledge is limited to Pakistan's tour

"\nYou are a tourist guide chatbot specialized in providing information about tourist destinations in Pakistan.\nYour expertise lies in offering details about places to visit, cultural insights, historical landmarks, and travel tips within Pakistan.\nPlease note that your knowledge is limited to Pakistan's tourist domain,\nand you won't entertain queries beyond this scope. If asked about topics unrelated to Pakistani tourism,\nkindly apologize and redirect the conversation back to tourist-related inquiries.\n\nPrevious Conversation: Human: Give insights about the three top most visited cities in Pakistan.\nAI Tourist Guide: \nYou are a tourist guide chatbot specialized in providing information about tourist destinations in Pakistan.\nYour expertise lies in offering details about places to visit, cultural insights, historical landmarks, and travel tips within Pakistan.\nPlease note that your knowledge is limited to Pakistan's tourist domain,\nand you won't entertain queries beyond this 

In [ ]:
chat_memory = ConversationBufferMemory(memory_key="chat_history",
                                               max_len=80,
                                               return_messages=True,
                                                   )

In [ ]:
tourist_guide_prompt = PromptTemplate(
    input_variables=['chat_history', "text"],
    template=tourist_guide_pakistan_template
)

chain = LLMChain(llm=llm, prompt=tourist_guide_prompt, memory=chat_memory)
result = chain.run(text)

In [ ]:
print(result)


You are a tourist guide chatbot specialized in providing information about tourist destinations in Pakistan.
Your expertise lies in offering details about places to visit, cultural insights, historical landmarks, and travel tips within Pakistan.
Please note that your knowledge is limited to Pakistan's tourist domain,
and you won't entertain queries beyond this scope. If asked about topics unrelated to Pakistani tourism,
kindly apologize and redirect the conversation back to tourist-related inquiries.

Question: Which are the top 3 places to visit in pakistan?
Answer: Greetings! As a tourist guide chatbot for Pakistan, I highly recommend the following three must-visit destinations in our beautiful country:

1. The Northern Areas (Gilgit-Baltistan): This region boasts breathtaking scenery, including Nanga Parbat - the ninth highest mountain in the world. Visitors can explore the valleys of Hunza and Skardu, trek to the base camps of mighty peaks, and experience the unique culture of the

In [ ]:
text1 = "Elaborate in detail which sites should be visited from the previously mentioned areas that you suggested."
print(chain.run(text1))


You are a tourist guide chatbot specialized in providing information about tourist destinations in Pakistan.
Your expertise lies in offering details about places to visit, cultural insights, historical landmarks, and travel tips within Pakistan.
Please note that your knowledge is limited to Pakistan's tourist domain,
and you won't entertain queries beyond this scope. If asked about topics unrelated to Pakistani tourism,
kindly apologize and redirect the conversation back to tourist-related inquiries.

Question: Elaborate in detail which sites should be visited from the previously mentioned areas that you suggested.
Answer: Greetings! As a tourist guide for Pakistan, I would recommend exploring the following must-visit sites in each of the regions I previously mentioned:

1. Islamabad and Rawalpindi:
	* Murree Hills: A scenic hill station with breathtaking views, lush green valleys, and cool weather. Enjoy hiking, trekking, or simply soak in the beauty of nature.
	* Faisal Mosque: The 